In [1]:
# Dependencies
import pandas as pd

### Transform the zip_to_zcta dataframe

In [2]:
# Import ZIP to ZCTA conversion table. All fields were imported as string to ensure the ZIP Codes retains the "00" prefixes 
zip_to_zcta_df = pd.read_excel("../00_input/zip_to_zcta_2019.xlsx", dtype = 'str')

In [3]:
# Check the ZIP/ZCTA dataframe data, particularly for retention of "00" for ZIP Codes
zip_to_zcta_df.head()

,ZIP_CODE,PO_NAME,STATE,ZIP_TYPE,ZCTA,zip_join_type
0,00501,Holtsville,NY,Post Office or large volume customer,11742,Spatial join to ZCTA
1,00544,Holtsville,NY,Post Office or large volume customer,11742,Spatial join to ZCTA
2,00601,Adjuntas,PR,Zip Code Area,00601,Zip Matches ZCTA
3,00602,Aguada,PR,Zip Code Area,00602,Zip Matches ZCTA
4,00603,Aguadilla,PR,Zip Code Area,00603,Zip Matches ZCTA


In [4]:
# Review data types in ZIP/ZCTA dataframe
zip_to_zcta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41107 entries, 0 to 41106
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ZIP_CODE       41107 non-null  object
 1   PO_NAME        41107 non-null  object
 2   STATE          41106 non-null  object
 3   ZIP_TYPE       41107 non-null  object
 4   ZCTA           41107 non-null  object
 5   zip_join_type  41107 non-null  object
dtypes: object(6)
memory usage: 1.9+ MB


In [5]:
# Remove unnecessary columns to refine dataset
zip_to_zcta_df.drop(columns=['PO_NAME', 'STATE', 'ZIP_TYPE', 'zip_join_type'], inplace=True)
zip_to_zcta_df.head()

,ZIP_CODE,ZCTA
0,00501,11742
1,00544,11742
2,00601,00601
3,00602,00602
4,00603,00603


In [6]:
# Check the minimum and maximum length of values in column ZIP_CODE
zip_code_len = zip_to_zcta_df["ZIP_CODE"].apply(len)
print(f"Max zip_code length is {zip_code_len.max()}.")
print(f"Min zip_code length is {zip_code_len.min()}.")

Max zip_code length is 5.
Min zip_code length is 5.


In [7]:
# Check the minimum and maximum length of values in column ZCTA
zcta_len = zip_to_zcta_df["ZCTA"].apply(len)
print(f"Max zcta length is {zcta_len.max()}.")
print(f"Min zcta length is {zcta_len.min()}.")

Max zcta length is 7.
Min zcta length is 5.


In [8]:
# A look into rows with length of values in column ZCTA > 5 
zip_to_zcta_df[zip_to_zcta_df.ZCTA.str.len() > zcta_len.min()]

,ZIP_CODE,ZCTA
41098,96939,No ZCTA
41099,96940,No ZCTA
41100,96941,No ZCTA
41101,96942,No ZCTA
41102,96943,No ZCTA
41103,96944,No ZCTA
41104,96960,No ZCTA
41105,96970,No ZCTA
41106,96898,No ZCTA


In [9]:
# Remove all rows with ZCTA = "No ZCTA"
zip_to_zcta_df = zip_to_zcta_df[~(zip_to_zcta_df.ZCTA.str.len() > zcta_len.min())]

In [10]:
zip_to_zcta_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41098 entries, 0 to 41097
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ZIP_CODE  41098 non-null  object
 1   ZCTA      41098 non-null  object
dtypes: object(2)
memory usage: 963.2+ KB


In [11]:
# Identify any duplicated ZIP Codes as all ZIP Codes should be unique (many ZIP Codes to one ZCTA relationship)
zip_to_zcta_df['ZIP_CODE'].nunique()

41098

### Test merging against zip_code in the restaurant table

In [12]:
# Import cleaned restaurant data
restaurant_clean_df = pd.read_csv("../02_transform_restaurant/restaurant_clean.csv", dtype = 'str')
restaurant_clean_df.head()

,restaurant_name,address,zip_code
0,SONIC Drive In,800 N Canal Blvd,70301
1,Taco Bell,206 Wears Valley Rd,37863
2,Arby's,3652 Parkway,37863
3,Steak 'n Shake,2118 Mt Zion Parkway,30260
4,Wendy's,9768 Grand River Ave,48204


In [13]:
# An overview of restaurant data
restaurant_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9343 entries, 0 to 9342
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   restaurant_name  9343 non-null   object
 1   address          9343 non-null   object
 2   zip_code         9343 non-null   object
dtypes: object(3)
memory usage: 219.1+ KB


In [14]:
# Merge restaurant data with zip_to_zcta data based on zip_code
restaurant_df_with_zcta = pd.merge(restaurant_clean_df, zip_to_zcta_df, left_on="zip_code", right_on="ZIP_CODE")
restaurant_df_with_zcta.head()

,restaurant_name,address,zip_code,ZIP_CODE,ZCTA
0,SONIC Drive In,800 N Canal Blvd,70301,70301,70301
1,Quiznos Sub,204 N Canal Blvd,70301,70301,70301
2,McDonald's,1020 S Acadia Rd,70301,70301,70301
3,Wingstop Restaurant,612 N Canal Blvd,70301,70301,70301
4,Taco Bell,206 Wears Valley Rd,37863,37863,37863


In [15]:
# Overview of restaurant data after merging --> all zip_codes in restaurant data have a matching ZCTA
restaurant_df_with_zcta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9343 entries, 0 to 9342
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   restaurant_name  9343 non-null   object
 1   address          9343 non-null   object
 2   zip_code         9343 non-null   object
 3   ZIP_CODE         9343 non-null   object
 4   ZCTA             9343 non-null   object
dtypes: object(5)
memory usage: 438.0+ KB


In [16]:
# Overview of rows with zip_codes different from ZCTA
restaurant_df_with_zcta[restaurant_df_with_zcta["zip_code"] != restaurant_df_with_zcta["ZCTA"]]

,restaurant_name,address,zip_code,ZIP_CODE,ZCTA
2997,B.GOOD,255 Washington St,02201,02201,02203
3322,Charley's Grilled Subs,1000 Airport Blvd,15231,15231,15108
4654,Arby's,4303 Prospect Ave,62524,62524,62526
5455,Chick-fil-A,800 S Gay St,37929,37929,37902
7456,Quiznos,1 Medical Dr,27599,27599,27514
8265,Church's Chicken,6425 Burnet Rd,73301,73301,78704
8283,McDonald's,250 Summit Park Dr,15275,15275,15108
8284,McDonald's,100 Davis Blvd,15275,15275,15108
8323,Arby's,787 Erie Blvd W,13201,13201,13202
8420,Subway,169 International Ctr,48824,48824,48825


In [ ]:
# break No and street in the address --> what are the streets that have the most restaurants
# Restaurant name --> table for restaurants -->
# 

### Export cleaned zip_to_zcta_df to csv

In [17]:
# Export ZIP_to_ZCTA dataframe into CSV file
zip_to_zcta_df.to_csv('zcta.csv', index=False)